In [1]:
import os
import sys
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image

import numpy as np
import tensorflow as tf
import pprint

In [2]:
tf.compat.v1.disable_eager_execution()

In [3]:
class CONFIG:
    IMAGE_WIDTH = 224
    IMAGE_HEIGHT = 224
    COLOR_CHANNELS = 3
    NOISE_RATIO = 0.6
    MEANS = np.array([129.1863, 104.7624, 93.5940]).reshape((1,1,1,3)) 
def load_vgg_model(path):
    """1 is conv1_1 (3, 3, 3, 64)
        2 is relu1_1
        3 is conv1_2 (3, 3, 64, 64)
        4 is relu1_2    
        5 is pool1
        6 is conv2_1 (3, 3, 64, 128)
        7 is relu2_1
        8 is conv2_2 (3, 3, 128, 128)
        9 is relu2_2
        10 is pool2
        11 is conv3_1 (3, 3, 128, 256)
        12 is relu3_1
        13 is conv3_2 (3, 3, 256, 256)
        14 is relu3_2
        15 is conv3_3 (3, 3, 256, 256)
        16 is relu3_3
        17 is pool3
        18 is conv4_1 (3, 3, 256, 512)
        19 is relu4_1
        20 is conv4_2 (3, 3, 512, 512)
        21 is relu4_2
        22 is conv4_3 (3, 3, 512, 512)
        23 is relu4_3
        24 is pool4
        25 is conv5_1 (3, 3, 512, 512)
        26 is relu5_1
        27 is conv5_2 (3, 3, 512, 512)
        28 is relu5_2
        29 is conv5_3 (3, 3, 512, 512)
        30 is relu5_3
        31 is pool5
        32 is fullyconnected (7, 7, 512, 4096)
        33 is relu6
        34 is fullyconnected (1, 1, 4096, 4096)
        35 is relu7
        36 is fullyconnected (1, 1, 4096, 2622)
        37 is softmax
    """
    
    vgg = scipy.io.loadmat(path)

    vgg_layers = vgg['layers']
    
    def _weights(layer, expected_layer_name):
        """
        Return the weights and bias from the VGG model for a given layer.
        """
        wb = vgg_layers[0][layer][0][0][2]
        W = wb[0][0]
        b = wb[0][1]
        layer_name = vgg_layers[0][layer][0][0][1][0]
        assert layer_name == expected_layer_name
        return W, b

    def _relu(conv2d_layer):
        """
        Return the RELU function wrapped over a TensorFlow layer. Expects a
        Conv2d layer input.
        """
        return tf.nn.relu(conv2d_layer)

    def _conv2d(prev_layer, layer, layer_name):
        """
        Return the Conv2D layer using the weights, biases from the VGG
        model at 'layer'.
        """
        W, b = _weights(layer, layer_name)
        W = tf.constant(W)
        b = tf.constant(np.reshape(b, (b.size)))
        return tf.nn.conv2d(prev_layer,W, strides=[1, 1, 1, 1], padding='SAME') + b
    def _conv2d1(prev_layer, layer, layer_name):
        """
        Return the Conv2D layer using the weights, biases from the VGG
        model at 'layer'.
        """
        W, b = _weights(layer, layer_name)
        W = tf.constant(W)
        b = tf.constant(np.reshape(b, (b.size)))
        return tf.nn.conv2d(prev_layer,W, strides=[1, 1, 1, 1], padding='VALID') + b

    def _conv2d_relu(prev_layer, layer, layer_name):
        """
        Return the Conv2D + RELU layer using the weights, biases from the VGG
        model at 'layer'.
        """
        return _relu(_conv2d(prev_layer, layer, layer_name))

    def _pool(prev_layer):
        """
        Return the MaxPooling layer.
        """
        return tf.nn.max_pool(prev_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Constructs the graph model.
    graph = {}
    graph['input']   = tf.Variable(np.zeros((1, CONFIG.IMAGE_HEIGHT, CONFIG.IMAGE_WIDTH, CONFIG.COLOR_CHANNELS)), dtype = 'float32')
    graph['conv1_1']  = _conv2d_relu(graph['input'], 0, 'conv1_1')
    graph['conv1_2']  = _conv2d_relu(graph['conv1_1'], 2, 'conv1_2')
    graph['pool1'] = _pool(graph['conv1_2'])
    graph['conv2_1']  = _conv2d_relu(graph['pool1'], 5, 'conv2_1')
    graph['conv2_2']  = _conv2d_relu(graph['conv2_1'], 7, 'conv2_2')
    graph['pool2'] = _pool(graph['conv2_2'])
    graph['conv3_1']  = _conv2d_relu(graph['pool2'], 10, 'conv3_1')
    graph['conv3_2']  = _conv2d_relu(graph['conv3_1'], 12, 'conv3_2')
    graph['conv3_3']  = _conv2d_relu(graph['conv3_2'], 14, 'conv3_3')
    graph['pool3'] = _pool(graph['conv3_3'])
    graph['conv4_1']  = _conv2d_relu(graph['pool3'], 17, 'conv4_1')
    graph['conv4_2']  = _conv2d_relu(graph['conv4_1'], 19, 'conv4_2')
    graph['conv4_3']  = _conv2d_relu(graph['conv4_2'], 21, 'conv4_3')
    graph['pool4'] = _pool(graph['conv4_3'])
    graph['conv5_1']  = _conv2d_relu(graph['pool4'], 24, 'conv5_1')
    graph['conv5_2']  = _conv2d_relu(graph['conv5_1'], 26, 'conv5_2')
    graph['conv5_3']  = _conv2d_relu(graph['conv5_2'], 28, 'conv5_3')
    graph['pool5'] = _pool(graph['conv5_3'])
    graph['fc6']  = _relu(_conv2d1(graph['pool5'], 31, 'fc6'))
    graph['fc7']  = _relu(_conv2d1(graph['fc6'], 33, 'fc7'))
    graph['fc8']  = _relu(_conv2d1(graph['fc7'], 35, 'fc8'))
    
    
    return graph




In [4]:
def reshape_and_normalize_image(image):
    """
    Reshape and normalize the input image (content or style)
    """
    
    # Reshape image to mach expected input of VGG16
    image = np.reshape(image, ((1,) + image.shape))
    
    # Substract the mean to match the expected input of VGG16
    #image = image - CONFIG.MEANS
    
    return image

In [56]:
def open_(path):
    image = Image.open(path)
    image = image.resize((224,224))
    image=np.array(image)
    image = reshape_and_normalize_image(image)
    return image

In [92]:
pp = pprint.PrettyPrinter(indent=4)
model = load_vgg_model("vgg-face.mat")
pp.pprint(model)

{   'conv1_1': <tf.Tensor 'Relu:0' shape=(1, 224, 224, 64) dtype=float32>,
    'conv1_2': <tf.Tensor 'Relu_1:0' shape=(1, 224, 224, 64) dtype=float32>,
    'conv2_1': <tf.Tensor 'Relu_2:0' shape=(1, 112, 112, 128) dtype=float32>,
    'conv2_2': <tf.Tensor 'Relu_3:0' shape=(1, 112, 112, 128) dtype=float32>,
    'conv3_1': <tf.Tensor 'Relu_4:0' shape=(1, 56, 56, 256) dtype=float32>,
    'conv3_2': <tf.Tensor 'Relu_5:0' shape=(1, 56, 56, 256) dtype=float32>,
    'conv3_3': <tf.Tensor 'Relu_6:0' shape=(1, 56, 56, 256) dtype=float32>,
    'conv4_1': <tf.Tensor 'Relu_7:0' shape=(1, 28, 28, 512) dtype=float32>,
    'conv4_2': <tf.Tensor 'Relu_8:0' shape=(1, 28, 28, 512) dtype=float32>,
    'conv4_3': <tf.Tensor 'Relu_9:0' shape=(1, 28, 28, 512) dtype=float32>,
    'conv5_1': <tf.Tensor 'Relu_10:0' shape=(1, 14, 14, 512) dtype=float32>,
    'conv5_2': <tf.Tensor 'Relu_11:0' shape=(1, 14, 14, 512) dtype=float32>,
    'conv5_3': <tf.Tensor 'Relu_12:0' shape=(1, 14, 14, 512) dtype=float32>,
    '

In [5]:
sess = tf.compat.v1.InteractiveSession()
model = load_vgg_model("vgg-face.mat")

In [58]:
sess.run(model['input'].assign(open_('images/DJ_Khaled.jpg')))
out=model['fc8']
DJ_Khaled=sess.run(out)
sess.run(model['input'].assign(open_('images/sunder.jpg')))
out=model['fc8']
sunder=sess.run(out)
sess.run(model['input'].assign(open_('images/selena.jpg')))
out=model['fc8']
selena=sess.run(out)
sess.run(model['input'].assign(open_('images/mark.jpg')))
out=model['fc8']
mark=sess.run(out)
sess.run(model['input'].assign(open_('images/elon.jpg')))
out=model['fc8']
elon=sess.run(out)

In [59]:
database = {}
database["DJ_Khaled"] = DJ_Khaled[0][0][0]
database["selena"] = selena[0][0][0]
database["mark"] = mark[0][0][0]
database["sunder"] = sunder[0][0][0]
database["elon"] = elon[0][0][0]

In [60]:
# GRADED FUNCTION: who_is_it

def who_is_it(image_path, database, model):
    
    sess.run(model['input'].assign(open_(image_path)))
    out=model['fc8']
    a=sess.run(out)
    min_dist = 10000
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        dist = np.linalg.norm(a[0][0][0]-db_enc)

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist<min_dist:
            min_dist = dist
            identity = name

    ### END CODE HERE ###
    
    print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [65]:
who_is_it("images/unknown4.jpg", database, model)

it's DJ_Khaled, the distance is 96.87281


(96.87281, 'DJ_Khaled')